#Second Practical work

This is second practical work skeleton.

It is divided in 3 parts_

1.   Python module load section: load all the needed modules
2.   Google Drive access and mounting
3.   Host load data
4.   GPU load





##Module load section
We will install the pycuda module in the colaboratory environment, to combine python and CUDA in our code. Because pycuda is not available by default, this task can take a few minutes.

As well we load numpy and matplotlib modules.

In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 37.0 MB/s 
  Installing build dependencies ... done


We import the Python modules which we will use in the notebook (numpy, pycuda and time module) 

In [ ]:
import  numpy  as  np
import  pycuda.autoinit
from    pycuda.compiler import SourceModule
import  pycuda.driver as  drv
import  pycuda.gpuarray as  gpuarray
from sys import getsizeof
import time

In [ ]:
%pylab inline
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile

##Mount Google Drive and access grant
We use Google API to allow access to our Google Drive.

For security and privacy reasons, this permission is temporally, and will be requested every time you restart the notebook. 

Your Google Drive will be mounted in the "/content/drive" folder. The first Folder will be "My Drive".

You have to copy your images to a folder in the "My Drive" folder in your Google Drive.

To check your available files, use the cell indicated as "**List Files in Folder**"

In [ ]:
from os.path import join
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
MYDRIVE="/content/drive/My Drive/"

**List Files in Folders**

In [ ]:
!ls /content/drive/My\ Drive/images

We define a variable IMAGE which is constructed joining the "MYDRIVE" varible plus the final path to your image.

In [ ]:
IMAGE=join(MYDRIVE,'images', 'chess.jpg')

In [ ]:
print("Image to load: {0}".format(IMAGE))

Loads an image in a numpy variable, and show.

In [ ]:
image=np.array(Image.open(IMAGE))
plt.figure()
plt.imshow(image)

Here we define the individual filters

In [ ]:
filter1=np.array([
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
])
filter2=np.array([[0.5, 0 , -0.5]])
filter3=np.array([[0.5],[0],[-0.5]])

filter4=np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]
])
filter5=np.array([
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]
])

Here we define the fiter masks, and gpu filter mask

In [ ]:
filter_mask=filter5.astype(np.float32)
filter_mask_gpu=gpuarray.to_gpu(filter_mask)

In [ ]:
image_shape=image.shape[0:2]

In [ ]:
block_s=16
BLOCK_SIZE=(block_s,block_s,1)
grid_size=(int(np.ceil(image_shape[1]/block_s)),int(np.ceil(image_shape[0]/block_s)))

#FILTER KERNEL

Here you have to define your image_filter GPU kernel.

In [ ]:
kernel_str = """
__global__ void image_filter( float * image,          //Source GPU array floating point 32 bits,
		                          float * filter_mask,    //Filter Mask GPU array 2D floating point 32 bits
                              float * filtered_image,  //Target GPU array 2D floating point 32 bits,
                              int NumRowsImg,         //Image Numrows,
                              int NumColsImg,         //Int32 Image Numcolumns,
                              int NumRowsFilter,       //Int32 Image NumRows filter mask,
                              int NumColsFilter       //Int32 Image NumCols filter mask
                              ) 
{ 
    //This instruction calculate in the offset in the memory data, based on the block identificator blockIdx.x
    // YOU HAVE TO WRITE HERE YOUR FILTER KERNEL USING TILED MEMORY ALGORITHMS
    int idxY;
    int idxX;
    int vX;

    uint tx = threadIdx.x;
    uint ty = threadIdx.y;
    idxY = blockIdx.y*blockDim.y+tx; //With this we calculate the row address in target matrix
    idxX = blockIdx.x*blockDim.x+ty;

    
    uint offset_x =  %(FILTER_COLS)s / 2;
    uint offset_y = %(FILTER_ROWS)s / 2;
    __shared__ float tile[%(TILE_ROWS)s][%(TILE_COLS)s];
    __shared__ float filter[%(FILTER_ROWS)s][ %(FILTER_COLS)s];
    if((idxX < NumColsImg) && (idxY < NumRowsImg) ) {
        // inside matrix
        tile[offset_y+ty][offset_x+tx] = image[idxY*NumColsImg+idxX];

    } else {

      uint temp_c = idxX;
      uint temp_r = idxY;
      if(idxX >= NumColsImg) {
        temp_c = NumColsImg-1; 
      }
      if(idxY >= NumRowsImg) {
        temp_r = NumRowsImg-1;
      }
      tile[offset_y+ty][offset_x+tx] = image[temp_r*NumColsImg+temp_c]; 
    }
    // left border
    if(tx<offset_x ) {
        int lbIdx_y = idxY;
        if(lbIdx_y>=NumRowsImg){
          lbIdx_y = NumRowsImg-1;
        }
        int lbIdx_x = idxX - offset_x;
        /*if(lbIdx_x >= NumColsImg) {
          lbIdx_x = NumColsImg-1;
        } else */
        if(lbIdx_x<0){
          // set flag for absolute border (use tile to fill it)
          lbIdx_x=0; // 0
        }
        // check corners here
        // top left corner
        
        if(ty < offset_y) {
          lbIdx_y -= offset_y;
          if (lbIdx_y < 0)
            lbIdx_y = 0;
           // have it come from tile
          tile[ty][tx]=image[lbIdx_y*NumColsImg+lbIdx_x];
        }
        // bottom left corner
         else if (ty >= blockDim.y-offset_y) {
          lbIdx_y += offset_y;
          //lbIdx_x -= offset_x;
          if(lbIdx_y >= NumRowsImg)
            lbIdx_y = NumRowsImg - 1;
          
          tile[ty+ 2*offset_y][tx]=image[lbIdx_y*NumColsImg+lbIdx_x];
        } else {
          tile[ty+offset_y][tx]=image[lbIdx_y*NumColsImg+lbIdx_x];
         // have it come from tile
         }
      } else if(tx>=blockDim.x-offset_x) {
        // right border
      int rbIdx_y = idxY;
        if(rbIdx_y>=NumRowsImg){
          // might not be correct if checking also for the bottom right corner
          rbIdx_y = NumRowsImg-1;
        }
        int rbIdx_x = idxX + offset_x;
        if(rbIdx_x >= NumColsImg) {
          // might not be correct if checking also for the top right corner
          rbIdx_x = NumColsImg-1;
        }
        // top right corner
        if (ty < offset_y) {
          rbIdx_x = idxX;
          rbIdx_y = idxY - offset_y;
          if (rbIdx_y < 0)
            rbIdx_y = 0;
          if (rbIdx_x >= NumColsImg)
            rbIdx_x = NumColsImg - 1;
          tile[ty][tx + 2*offset_x]=image[rbIdx_y*NumColsImg+rbIdx_x];
          // bottom right corner
        } else if(ty>=(blockDim.y-offset_y)){
          rbIdx_y += offset_y;
          if(rbIdx_y>=NumRowsImg)
            rbIdx_y=NumRowsImg-1;
          tile[tx+2*offset_y][tx+2*offset_x]=image[rbIdx_y*NumColsImg+rbIdx_x];

        } else {

        
          tile[ty+offset_y][tx+2*offset_x]=image[rbIdx_y*NumColsImg+rbIdx_x];
        }
    } 
    
     // bottom border

    else if (ty>= blockDim.y-offset_y) {
      int bbIdx_x = idxX;
      if (bbIdx_x >= NumColsImg){
        bbIdx_x = NumColsImg-1;
      }
      int bbIdx_y = idxY+offset_y;
      if(bbIdx_y>=NumRowsImg) {
        bbIdx_y = NumRowsImg-1;
      }
      tile[ty+2*offset_y][tx+offset_x] = image[bbIdx_y*NumColsImg+bbIdx_x];
    } else if (ty < offset_y) {
      //  top border
      int tbIdx_x = idxX;
      if (tbIdx_x >= NumColsImg){
        tbIdx_x = NumColsImg-1;
      }
      int tbIdx_y = idxY-offset_y;
      if(tbIdx_y<0){
        tbIdx_y = 0;
      }
      tile[ty][tx+offset_x] = image[tbIdx_y*NumColsImg+tbIdx_x];
    }
    
    if(tx<NumColsFilter && ty < NumRowsFilter) {
      filter[ty][tx] = filter_mask[ty*NumColsFilter+tx];
    }
    __syncthreads();

    //THIS CODE JUST COPY THE SOURCE IMAGE IN THE TARGET VECTOR
    float acc = 0;
    if ((idxY<NumRowsImg) && (idxX<NumColsImg)){
      for(int row=0; row<NumRowsFilter; row++) {
        for(int col=0; col<NumColsFilter; col++) {
          /*
          if (ty<offset_y) {
            // top border
            if (tx < offset_x)
              // top left corner
              acc += tile[ty+row+offset_y][tx+col+offset_x] * filter[row][col];
            else if (tx >= blockDim.x-offset_x)
            // top right
              acc += tile[ty+row+offset_y][tx+col-offset_x] * filter[row][col];
            else
              acc += tile[ty+row+offset_y][tx+col] * filter[row][col];
          } else if(ty >= blockDim.y-offset_y) {
            // bottom
            if (tx < offset_x)
              // bottom left corner
              acc += tile[ty+row-offset_y][tx+col+offset_x] * filter[row][col];
            else if (tx >= blockDim.x-offset_x)
            // top right
              acc += tile[ty+row-offset_y][tx+col-offset_x] * filter[row][col];
            else
              acc += tile[ty+row- offset_y][tx+col] * filter[row][col];
          } else if (tx < offset_x) {
            //left
            acc += tile[ty+row][tx+col+ offset_x] * filter[row][col];
          }
          else if(tx >= blockDim.x-offset_x) {
            // right
            acc += tile[ty+row][tx+col- offset_x] * filter[row][col];
          }
          else*/
          acc+=tile[ty+row][tx+col] * filter[row][col];

        }
      }
      vX=idxY*NumColsImg+idxX;
      filtered_image[vX] = acc;
    }


    __syncthreads();
}

"""

In [ ]:
FILTER_ROWS = filter_mask.shape[0]
FILTER_COLS = filter_mask.shape[1]
TILE_ROWS = block_s + FILTER_ROWS - 1
TILE_COLS = block_s + FILTER_COLS -1

In [ ]:
kernel_src = kernel_str % {
    'BLOCK_SIZE': BLOCK_SIZE,
    'FILTER_ROWS': FILTER_ROWS,
    'FILTER_COLS': FILTER_COLS,
    'TILE_ROWS': TILE_ROWS,
    'TILE_COLS':TILE_COLS
}

In [ ]:
kernel = SourceModule(kernel_src)

And import the kernel definition to python

In [ ]:
image_filter = kernel.get_function ('image_filter')

Recover the image shape, and select the two first items (rows and columns).

Set block size and grid size.

Allocates the target image memory.

In [ ]:
result_image=np.zeros(image.shape,dtype=np.uint8)

In [ ]:
depth = image.shape[2]

Here we defines our sample filter masks.

We select the filter mask, and upload to the GPU device.

##Prepares the execution environment 

Allocate the temporal filtered memory layer, where will be store the filtered image.

In [ ]:
filtered_image_layer_gpu = gpuarray.zeros(image_shape,np.float32)

Here defines the main block.
Walk through the layers. Copying the selected layer to temporal image_layer, upload the layer to the GPU and invokes the image_filter kernel.

In [ ]:
start_t = time.time()
for layer in range(depth):
  image_layer = np.zeros(image_shape,dtype=np.float32) #Allocate the temporal image layer
  image_layer=(image[:,:,layer]).astype(np.float32)    #Copy from the orignal image the selected layer into the temporal memory
  image_layer_gpu=gpuarray.to_gpu(image_layer)         #Uploads the image layer to the GPU
  filtered_image_layer_gpu.fill(0.0)                   #Fills with 0 the target memory in the GPU
  image_filter(                                        #invokes the kernel
          image_layer_gpu,
          filter_mask_gpu,
          filtered_image_layer_gpu,
          np.int32(image_shape[0]),
          np.int32(image_shape[1]),
          np.int32(filter_mask.shape[0]),
          np.int32(filter_mask.shape[1]),
          block=BLOCK_SIZE,
          grid=grid_size
        )
  filtered_image_layer=filtered_image_layer_gpu.get() #Download from the GPU the filtered image
  result_image[:,:,layer] = filtered_image_layer.astype(np.uint8) #Copy the filtered image to the final image store, 
                                                                  #casting the data type from double to uint8
end_t = time.time()

#Show results


In [ ]:

print("Execution time is: {0} seconds.".format(end_t-start_t))

In [ ]:
plt.figure()
plt.imshow(result_image)